In [1]:
import os
os.chdir("C:/Users/Anushka/Desktop/anushka/ML/projects_vscode/IPL prediction system")

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
import lightgbm as lgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
import os

In [3]:
df = pd.read_csv("data_processing/processed_data/match_level_data.csv")

In [4]:
X = df[["team1_score", "team1_wickets"]]
y = df["winner_encoded"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")
xgb.fit(X_train, y_train)
xgb_preds = xgb.predict(X_test)
print("\n[XGBoost] Accuracy:", accuracy_score(y_test, xgb_preds))

# LightGBM
lgbm = lgb.LGBMClassifier()
lgbm.fit(X_train, y_train)
lgbm_preds = lgbm.predict(X_test)
print("\n[LightGBM] Accuracy:", accuracy_score(y_test, lgbm_preds))

# Neural Network
nn = Sequential()
nn.add(Dense(32, input_dim=X.shape[1], activation='relu'))
nn.add(Dense(64, activation='relu'))
nn.add(Dense(len(np.unique(y)), activation='softmax'))
nn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
nn.fit(X_train, y_train, epochs=50, batch_size=8, verbose=0)
nn_loss, nn_acc = nn.evaluate(X_test, y_test, verbose=0)
print("\n[Neural Net] Accuracy:", nn_acc)

c:\Users\Anushka\Desktop\anushka\ML\projects_vscode\IPL prediction system\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [03:44:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



[XGBoost] Accuracy: 0.10550458715596331
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000438 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 133
[LightGBM] [Info] Number of data points in the train set: 872, number of used features: 2
[LightGBM] [Info] Start training from score -2.145817
[LightGBM] [Info] Start training from score -3.474953
[LightGBM] [Info] Start training from score -3.081910
[LightGBM] [Info] Start training from score -2.745438
[LightGBM] [Info] Start training from score -4.691348
[LightGBM] [Info] Start training from score -3.826350
[LightGBM] [Info] Start training from score -2.480330
[LightGBM] [Info] Start training from score -5.384495
[LightGBM] [Info] Start training from score -2.145817
[LightGBM] [Info] Start training from score -3.726267
[LightGBM] [Info] Start training from score -2.008615
[LightGBM] [Info

c:\Users\Anushka\Desktop\anushka\ML\projects_vscode\IPL prediction system\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



[Neural Net] Accuracy: 0.16055046021938324


In [7]:
# Save models
os.makedirs("ml_model", exist_ok=True)
joblib.dump(xgb, "ml_model/xgb_match_winner.pkl")
joblib.dump(lgbm, "ml_model/lgbm_match_winner.pkl")
nn.save("ml_model/nn_match_winner.h5")